<a href="https://colab.research.google.com/github/sonomarina/Stock-sentiment-analysis/blob/main/F%26M_EX1_Model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports from the functions given to us 

In [ ]:

import helpFunctions as hf
import matplotlib.pyplot as plt
import numpy as np
import imageio


Day one multispectracl image

In [ ]:

multiIm, annotationIm = hf.loadMulti('multispectral_day01.mat' ,\
'annotation_day01.png', dirIn)




EXTRACT PIXELS RELATED TO FAT AND MEAT

In [ ]:
[fatPix, fatR, fatC] = hf.getPix(multiIm, annotationIm[:,:,1]);
[meatPix, meatR, meatC] = hf.getPix(multiIm, annotationIm[:,:,2]);

Normal distribution PDF 

In [ ]:

def pdf(x, mean, var):
    return 1/np.sqrt(var)/np.sqrt(2*np.pi)*np.exp(-(x-mean)**2/2/var)
# OBSERVATIONS
x = np.linspace(0, 100, 1000)
# THRESHOLDS FOR ALL SPECTRAL BANDS
cutoffs = np.zeros(19)
meatMean = np.mean(meatPix,0)
fatMean = np.mean(fatPix,0)
for i in range(0, 19):
# COMMON VARIANCE
    var = ((len(meatPix[:,i]) - 1)*np.var(meatPix[:,i]) + (len(fatPix[:,i]) - \1)*np.var(fatPix[:,i]))/(len(meatPix[:,i]) + len(fatPix[:,i]) - 2) 
  f_meat = pdf(x, meatMean[i], var)
  f_fat = pdf(x, fatMean[i], var)
  xmax = x[f_fat - f_meat >= 0]
  cutoffs[i] = xmax[0]

In [ ]:
# IDENTIFIED CLASSIFICATIONS VS KNOWN CLASSIFICATIONS
rate = np.zeros(19)
for i in range(0, 19):
meat_rate = fatPix[:,i] - cutoffs[i]
fat_rate = cutoffs[i] - meatPix[:,i]
rate[i] = len(meat_rate[meat_rate > 0]) + len(fat_rate[fat_rate > 0])
# BAND WITH THE MOST CORRECT CLASSIFICATIONS
bandNr = np.where(rate == max(rate)) # BAND NR. 14 and 15 (863 nm AND 880 nm)
# CLASSIFY BASED ON BAND 14 FOR VISUALISATION
threshold = cutoffs[13]
multiIm[multiIm < threshold] = 0
multiIm[multiIm >= threshold] = 1
# MASK FOR VISUALISATION
mask = annotationIm[:,:,1]
mask[multiIm[:,:,13] == 0] = False # USE BAND 13 DATA FOR THE MASK
mask[multiIm[:,:,13] == 1] = True
bPix = np.where(np.logical_and(annotationIm[:,:,0] == 0, \
annotationIm[:,:,1] == 0, annotationIm[:,:,2] == 0))
mask[bPix] = 0
# CLASSIFIED IMAGE PLOT
[fatPix, fatR, fatC] = hf.getPix(multiIm, mask)
pixId = np.stack((fatR, fatC), axis=1)
imRGB = imageio.imread(dirIn + 'color_day01.png')
rgbOut = hf.setImagePix(imRGB, pixId)
plt.imshow(rgbOut)
plt.title('Classified image of the salami on day 1')
plt.xlim(0,514)
plt.ylim(514,0)
plt.savefig('salami_day1_model_1', dpi = 300)
plt.show()